In [1]:
data_path = "src/resources/data/insurance.csv"

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("customer_look_alike_modelling").getOrCreate()
df = spark.read.option("header", "true").csv(data_path)
df.limit(5).show()

24/08/26 13:54:09 WARN Utils: Your hostname, dvadmin-ThinkPad-E480 resolves to a loopback address: 127.0.1.1; using 192.168.0.15 instead (on interface wlp5s0)
24/08/26 13:54:09 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/08/26 13:54:19 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/08/26 13:54:24 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


+---+---+------+------+--------+------+---------+--------------------+
| Id|age|   sex|   bmi|children|smoker|   region|charges_bucket_label|
+---+---+------+------+--------+------+---------+--------------------+
|  0| 19|female|  27.9|       0|   yes|southwest|           15k - 20k|
|  1| 18|  male| 33.77|       1|    no|southeast|            Under 5k|
|  2| 28|  male|  33.0|       3|    no|southeast|            Under 5k|
|  3| 33|  male|22.705|       0|    no|northwest|           20k - 25k|
|  4| 32|  male| 28.88|       0|    no|northwest|            Under 5k|
+---+---+------+------+--------+------+---------+--------------------+



In [6]:
df.columns

['Id',
 'age',
 'sex',
 'bmi',
 'children',
 'smoker',
 'region',
 'charges_bucket_label']

In [7]:
rows_to_convert = df.columns
rows_to_convert.remove("Id")
rows_to_convert

['age', 'sex', 'bmi', 'children', 'smoker', 'region', 'charges_bucket_label']

In [8]:
",".join(rows_to_convert)

'age,sex,bmi,children,smoker,region,charges_bucket_label'

In [9]:
df.count()

1338

In [10]:
from src.utils.functions import get_row_as_text, hf_embeddings, get_ars_retrieved_df
train_df = get_row_as_text(df, rows_to_convert)

/home/dvadmin/look_alike_modelling_streamlit/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/dvadmin/look_alike_modelling_streamlit/venv/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [11]:
train_df.select("row_as_text").show(2, truncate=False)

+-------------------------------------------------------------------------------------------------------------+
|row_as_text                                                                                                  |
+-------------------------------------------------------------------------------------------------------------+
|age: 19; sex: female; bmi: 27.9; children: 0; smoker: yes; region: southwest; charges_bucket_label: 15k - 20k|
|age: 18; sex: male; bmi: 33.77; children: 1; smoker: no; region: southeast; charges_bucket_label: Under 5k   |
+-------------------------------------------------------------------------------------------------------------+
only showing top 2 rows



In [12]:
train_df.printSchema()

root
 |-- Id: long (nullable = false)
 |-- age: string (nullable = true)
 |-- sex: string (nullable = true)
 |-- bmi: string (nullable = true)
 |-- children: string (nullable = true)
 |-- smoker: string (nullable = true)
 |-- region: string (nullable = true)
 |-- charges_bucket_label: string (nullable = true)
 |-- row_as_text: string (nullable = false)



In [13]:
step = 1500
k = 4000

texts_list = train_df.rdd.collect()
# texts_list = [x[0] for x in texts_label_list]
# texts_list
texts_list[0]

Row(Id=0, age='19', sex='female', bmi='27.9', children='0', smoker='yes', region='southwest', charges_bucket_label='15k - 20k', row_as_text='age: 19; sex: female; bmi: 27.9; children: 0; smoker: yes; region: southwest; charges_bucket_label: 15k - 20k')

In [14]:
import os

# Specify the path of the directory you want to create
db_dir = "src/resources/embeddings/insurance"

# Create the directory
os.makedirs(db_dir, exist_ok=True)

In [66]:
from langchain.vectorstores import Chroma
from src.utils.functions import hf_embeddings
import chromadb
import os

client = chromadb.PersistentClient(path=db_dir)

vdb = Chroma(persist_directory=db_dir, embedding_function=hf_embeddings,
                 collection_metadata={"hnsw:space": "cosine"}, client=client)

In [21]:
from langchain.vectorstores import Chroma
vdb = Chroma(persist_directory=db_dir, embedding_function=hf_embeddings, collection_metadata={"hnsw:space": "cosine"})
for i in range(0, len(texts_list), step):
    texts = [x.row_as_text for x in texts_list[i:i+step]]
    metadata = [{"Id": str(x.Id), "charges_bucket_label": str(x.charges_bucket_label)} for x in texts_list[i:i+step]]
    vdb.add_texts(texts, metadata)

vdb.persist()

In [ ]:
from langchain.vectorstores import Chroma
from src.utils.functions import hf_embeddings
import chromadb

client = chromadb.PersistentClient(path=db_dir)

vdb = Chroma(persist_directory=db_dir, embedding_function=hf_embeddings,
                 collection_metadata={"hnsw:space": "cosine"}, client=client)

In [20]:
vdb._collection.count()

1338